In [1]:
def extract_to_df(url):
    '''
    TODO - write docstring
    '''

    # Request data from the API
    response = requests.get(url)

    # Convert the response to JSON
    response_text = response.text

    # Convert JSON to a Python data type (lists/dictionaries)
    results_list = json.loads(response_text)

    # Convert the Python lists/dictionaries to a Pandas DataFrame
    df = pandas.DataFrame(results_list)

    # Return the DataFrame
    return df


In [2]:
import pandas
import requests
import json
import datetime
import arcgis

# base url for the data
url = "https://data.sfgov.org/resource/vw6y-z8j6.json"

# build the field list
field_list = [
    'service_request_id',
    'requested_datetime',
    'status_notes',
    'service_name',
    'service_subtype',
    'lat',
    'long',
    'neighborhoods_sffind_boundaries',
    'source',
    'supervisor_district',
    'media_url',
    'point'
]
fields = "$select=" + ",".join(field_list)

# build the where statement
now = datetime.datetime.now()
start_date = now - datetime.timedelta(days=7)

# Set the time to midnight
midnight = datetime.time()

# Combine the date and time
start_day_midnight = datetime.datetime.combine(start_date, midnight)
start_day_string = start_day_midnight.strftime('%Y-%m-%dT%H:%M:%S.%f') 
where = f"$where=status_description='Open' and requested_datetime > '{start_day_string}'"

# set a record limit
limit = "&$limit=10000"

# combine the url components
full_url = url+"?" + fields + "&" + where + limit

# call our function to extract and convert to a DataFrame
df = extract_to_df(full_url)

In [ ]:
# Review the results
print(df.shape)
df.head()

In [ ]:
def street_cleaning(value):
    if value == 'Street and Sidewalk Cleaning':
        return 1
    else:
        return 0
    
print(street_cleaning('Street and Sidewalk Cleaning'))
print(street_cleaning('some other string'))

In [ ]:
lambda_func = lambda x: 1 if x == 'Street and Sidewalk Cleaning' else 0

print(lambda_func('Street and Sidewalk Cleaning'))
print(lambda_func('some other string'))

In [ ]:
df['service_name'].apply(
    lambda r: 1 if r== 'Street and Sidewalk Cleaning' else 0
)

In [7]:
# street/sidewalk cleaning yes/no
df['street_sidewalk_cleaning'] = df['service_name'].apply(
    lambda r: 1 if r== 'Street and Sidewalk Cleaning' else 0
)

# graffiti yes/no
df['graffiti'] = df['service_name'].apply(
    lambda r: 1 if r== 'Graffiti' else 0
)

# counter column for total cases
df['total_cases'] = 1

In [ ]:
df[['service_name','street_sidewalk_cleaning','graffiti','total_cases']]

In [ ]:
df_neighborhood = df.groupby("neighborhoods_sffind_boundaries").agg(
    {
        "total_cases": "sum",
        "street_sidewalk_cleaning": "sum",
        "graffiti": "sum"
    }
)
df_neighborhood

In [10]:
sedf_neighborhoods = pandas.DataFrame.spatial.from_featureclass(
    "../Chapter 09/Tutorial_09.gdb/SF_Find_Neighborhoods"
)

In [11]:
sedf_merge = sedf_neighborhoods.merge(
    df_neighborhood, 
    how = 'inner', 
    left_on = 'name', 
    right_on = 'neighborhoods_sffind_boundaries'
)

In [ ]:
sedf_merge.spatial.to_featureclass(
    "../Chapter 09/Tutorial_09.gdb/sf_311_cases_prev_7_days"
)

Copyright 2025 Esri